In [7]:
import db_utils
import yaml, pandas as pd, numpy as np

with open('credentials.yaml') as file:
    credentials_dict = yaml.safe_load(file)
    
credentials = db_utils.RDSDatabaseConnector(credentials_dict)
loan_payments = credentials.initialise_database()

credentials.save_file(loan_payments, 'new_file.csv')
database = pd.read_csv('new_file.csv', index_col = 'id')

cleaned_data = db_utils.DataTransform(database)

date_data = ['issue_date', 'earliest_credit_line', 'last_payment_date', 'next_payment_date',
             'last_credit_pull_date']

categorical_data = ['member_id', 'term', 'int_rate', 'grade', 'sub_grade', 'employment_length', 'home_ownership', 'verification_status', 'loan_status', 
                    'payment_plan', 'purpose', 'policy_code', 'application_type']

non_numeric_data = date_data + categorical_data 
column_headings = database.columns.values.tolist()
numeric_data = [column for column in column_headings if column not in non_numeric_data]

for date_column in date_data:
    database, date_column = cleaned_data.date_data(database, date_column)

for categories in categorical_data:
    database, categories = cleaned_data.change_type(database, categories, 'category')

df_info = db_utils.DataFrameInfo(database)

for column in column_headings:
    null_vals, null_percentage = df_info.missing(database, column)
    null_percentage = round(null_percentage, 2)
    if null_percentage > float(50):
        database = database.drop(column, axis=1)
    elif float(0) < null_percentage < float(50):
        print(f'{column}: {null_vals} null values, {null_percentage}%')

funded_amount_column = list(database['funded_amount'])
loan_amount_column = list(database['loan_amount'])

database = database.sort_values(by=['int_rate'])
int_rate_values = df_info.unique_vals(database, 'int_rate')
print(int_rate_values)

for index in range(54231):
    if np.isnan(funded_amount_column[index]):
        funded_amount_column[index] = loan_amount_column[index]

database['funded_amount'] = funded_amount_column

credentials.save_file(database, 'new_file_2.csv')




TypeError: RDSDatabaseConnector.save_file() takes 2 positional arguments but 3 were given